# How to write clean and maintainable code with Dapper

Writing clean and maintainable code with Dapper is a skill that can help you create efficient and robust data access layers for your applications. Dapper is a micro-ORM (object-relational mapper) that allows you to map your database tables and columns to your C# classes and properties. It also provides a simple and fast way to execute SQL queries and commands.
To write clean and maintainable code with Dapper, you can follow some of these tips and best practices:
* Use __modular code__ that is easy to maintain and test. Break down your code into smaller, reusable modules that can be used across your codebase. Use interfaces and abstractions to define the interactions between modules.
* Use __meaningful names__ for your classes, properties, methods, variables, and parameters. Avoid using abbreviations, acronyms, or generic names that can cause confusion or ambiguity. Use consistent naming conventions and follow the C# coding standards.
* Use __concise and clear code__ that expresses your intent and logic. Avoid unnecessary complexity, redundancy, or noise words that can make your code hard to read or understand. Use comments and documentation to explain the purpose and functionality of your code.
* Use __reusable code__ that avoids duplication and promotes consistency. Use Dapper's extension methods, query builders, and custom mappers to simplify your data access code and reduce boilerplate code. Use Dapper's multi-mapping feature to map multiple objects from a single query.
* Use __clear flow of execution__ that follows the single responsibility principle. Each method or module should have one and only one reason to change. Use Dapper's async and await keywords to perform asynchronous data access operations and avoid blocking the main thread.

To illustrate these tips and best practices, let's assume that we want to create a data access layer for the Product table, which contains information about the products that the company sells. The Product table has the following schema:
| Column Name | Data Type | Description |
| --- | --- | --- |
| ProductID | int | The primary key for the Product table |
| Name | nvarchar(50) | The name of the product |
| ProductNumber | nvarchar(25) | The unique product number |
| Color | nvarchar(15) | The color of the product |
| StandardCost | money | The standard cost of the product |
| ListPrice | money | The selling price of the product |
| Size | nvarchar(5) | The size of the product |
| Weight | decimal(8,2) | The weight of the product |
| ProductCategoryID | int | The foreign key to the ProductCategory table |
| ProductModelID | int | The foreign key to the ProductModel table |
| SellStartDate | datetime | The date when the product was available for sale |
| SellEndDate | datetime | The date when the product was no longer available for sale |
| DiscontinuedDate | datetime | The date when the product was discontinued |
| rowguid | uniqueidentifier | The row identifier for the table |
| ModifiedDate | datetime | The date when the row was last updated |

The first step to write clean and maintainable code with Dapper is to create a C# class that represents the Product entity. This class will map the columns of the Product table to its properties. We can use the following code to create the Product class:

In [1]:
// Connection string
#load "AppSettings.cs"

In [2]:
#r "nuget:Microsoft.Data.SqlClient"
#r "nuget:Dapper"

Installed Packages Dapper, 2.1.24 Microsoft.Data.SqlClient, 5.1.2

In [3]:
using System;

// A class that represents the Product entity
public class Product
{
    public int ProductID { get; set; }
    public required string Name { get; set; }
    public required string ProductNumber { get; set; }
    public required string Color { get; set; }
    public decimal StandardCost { get; set; }
    public decimal ListPrice { get; set; }
    public required string Size { get; set; }
    public decimal? Weight { get; set; }
    public int? ProductCategoryID { get; set; }
    public int? ProductModelID { get; set; }
    public DateTime SellStartDate { get; set; }
    public DateTime? SellEndDate { get; set; }
    public DateTime? DiscontinuedDate { get; set; }
    public Guid rowguid { get; set; }
    public DateTime ModifiedDate { get; set; }
}

Notice how we use meaningful names for the class and its properties, and how we use comments to explain the purpose and functionality of each property. We also use the appropriate data types for each property, and we use nullable types for the columns that can have null values.
The next step is to create an interface that defines the contract for the data access operations that we want to perform on the Product table. This interface will abstract the implementation details of the data access layer and will allow us to use dependency injection and unit testing. We can use the following code to create the <code>IProductRepository</code> interface:

In [4]:
using System.Collections.Generic;
using System.Threading.Tasks;

// An interface that defines the contract for the data access operations on the Product table
public interface IProductRepository
{
    // A method that returns all the products
    Task<IEnumerable<Product>> GetAllProductsAsync();

    // A method that returns a product by its ID
    Task<Product> GetProductByIdAsync(int productId);

    // A method that returns the products by their category ID
    Task<IEnumerable<Product>> GetProductsByCategoryIdAsync(int categoryId);

    // A method that returns the products by their model ID
    Task<IEnumerable<Product>> GetProductsByModelIdAsync(int modelId);

    // A method that adds a new product
    Task<int> AddProductAsync(Product product);

    // A method that updates an existing product
    Task<int> UpdateProductAsync(Product product);

    // A method that deletes an existing product
    Task<int> DeleteProductAsync(int productId);
}

Notice how we use meaningful names for the interface and its methods, and how we use comments to explain the purpose and functionality of each method. We also use the async and await keywords to perform asynchronous data access operations, and we use the Task<T> type to return the results of the operations.
The final step is to create a class that implements the IProductRepository interface and uses Dapper to execute the SQL queries and commands on the Product table. This class will contain the actual data access logic and will use Dapper's extension methods, query builders, and custom mappers to simplify the code and reduce boilerplate code. We can use the following code to create the ProductRepository class:

In [5]:
using System.Collections.Generic;
using Microsoft.Data.SqlClient;
using System.Data;
using System.Linq;
using System.Threading.Tasks;
using Dapper;

// A class that implements the IProductRepository interface and uses Dapper to execute the SQL queries and commands on the Product table
public class ProductRepository : IProductRepository
{
    // The connection string to the AdventureWorks2022 database
    private readonly string _connectionString;

    // The constructor that takes the connection string as a parameter
    public ProductRepository(string connectionString)
    {
        _connectionString = connectionString;
    }

    // A method that returns all the products
    public async Task<IEnumerable<Product>> GetAllProductsAsync()
    {
        // Create a SQL query to select all the products
        var sql = @"SELECT * FROM Production.Product";

        // Create a connection to the database
        using (var connection = new SqlConnection(_connectionString))        
        {
            // Open the connection
            await connection.OpenAsync();

            // Execute the query and return the results as a list of Product objects
            return await connection.QueryAsync<Product>(sql);
        }
    }

    // A method that returns a product by its ID
    public async Task<Product> GetProductByIdAsync(int productId)
    {
        // Create a SQL query to select a product by its ID
        var sql = @"SELECT * FROM Production.Product WHERE ProductID = @ProductID";

        // Create a dynamic parameter object with the product ID
        var parameters = new DynamicParameters();
        parameters.Add("@ProductID", productId);

        // Create a connection to the database
        using (var connection = new SqlConnection(_connectionString))
        {
            // Open the connection
            await connection.OpenAsync();

            // Execute the query and return the result as a Product object
            return await connection.QueryFirstOrDefaultAsync<Product>(sql, parameters);
        }
    }

    // A method that returns the products by their category ID
    public async Task<IEnumerable<Product>> GetProductsByCategoryIdAsync(int categoryId)
    {
        // Create a SQL query to select the products by their category ID
        var sql = @"SELECT p.* FROM Production.Product p
                    INNER JOIN Production.ProductCategory pc
                    ON p.ProductCategoryID = pc.ProductCategoryID
                    WHERE pc.ProductCategoryID = @ProductCategoryID";

        // Create a dynamic parameter object with the category ID
        var parameters = new DynamicParameters();
        parameters.Add("@ProductCategoryID", categoryId);

        // Create a connection to the database
        using (var connection = new SqlConnection(_connectionString))
        {
            // Open the connection
            await connection.OpenAsync();

            // Execute the query and return the results as a list of Product objects
            return await connection.QueryAsync<Product>(sql, parameters);
        }
    }

    // A method that returns the products by their model ID
    public async Task<IEnumerable<Product>> GetProductsByModelIdAsync(int modelId)
    {
        // Create a SQL query to select the products by their model ID
        var sql = @"SELECT p.* FROM Production.Product p
                    INNER JOIN Production.ProductModel pm
                    ON p.ProductModelID = pm.ProductModelID
                    WHERE pm.ProductModelID = @ProductModelID";

        // Create a dynamic parameter object with the model ID
        var parameters = new DynamicParameters();
        parameters.Add("@ProductModelID", modelId);

        // Create a connection to the database
        using (var connection = new SqlConnection(_connectionString))
        {
            // Open the connection
            await connection.OpenAsync();

            // Execute the query and return the results as a list of Product objects
            return await connection.QueryAsync<Product>(sql, parameters);
        }
    }

    // A method that adds a new product
    public async Task<int> AddProductAsync(Product product)
    {
        // Create a SQL command to insert a new product
        var sql = @"INSERT INTO Production.Product
                    (Name, ProductNumber, Color, StandardCost, ListPrice, Size, Weight, ProductCategoryID, ProductModelID, SellStartDate, SellEndDate, DiscontinuedDate, rowguid, ModifiedDate)
                    VALUES
                    (@Name, @ProductNumber, @Color, @StandardCost, @ListPrice, @Size, @Weight, @ProductCategoryID, @ProductModelID, @SellStartDate, @SellEndDate, @DiscontinuedDate, @rowguid, @ModifiedDate);
                    SELECT SCOPE_IDENTITY();";

        // Create a dynamic parameter object with the product properties
        var parameters = new DynamicParameters();
        parameters.Add("@Name", product.Name);
        parameters.Add("@ProductNumber", product.ProductNumber);
        parameters.Add("@Color", product.Color);
        parameters.Add("@StandardCost", product.StandardCost);
        parameters.Add("@ListPrice", product.ListPrice);
        parameters.Add("@Size", product.Size);
        parameters.Add("@Weight", product.Weight);
        parameters.Add("@ProductCategoryID", product.ProductCategoryID);
        parameters.Add("@ProductModelID", product.ProductModelID);
        parameters.Add("@SellStartDate", product.SellStartDate);
        parameters.Add("@SellEndDate", product.SellEndDate);
        parameters.Add("@DiscontinuedDate", product.DiscontinuedDate);
        parameters.Add("@rowguid", product.rowguid);
        parameters.Add("@ModifiedDate", product.ModifiedDate);

        // Create a connection to the database
        using (var connection = new SqlConnection(_connectionString))
        {
            // Open the connection
            await connection.OpenAsync();

            // Execute the command and return the result as the new product ID
            return await connection.ExecuteScalarAsync<int>(sql, parameters);
        }
    }

    // A method that updates an existing product
    public async Task<int> UpdateProductAsync(Product product)
    {
        // Create a SQL command to update an existing product
        var sql = @"UPDATE Production.Product
                    SET Name = @Name,
                    ProductNumber = @ProductNumber,
                    Color = @Color,
                    StandardCost = @StandardCost,
                    ListPrice = @ListPrice,
                    Size = @Size,
                    Weight = @Weight,
                    ProductCategoryID = @ProductCategoryID,
                    ProductModelID = @ProductModelID,
                    SellStartDate = @SellStartDate,
                    SellEndDate = @SellEndDate,
                    DiscontinuedDate = @DiscontinuedDate,
                    rowguid = @rowguid,
                    ModifiedDate = @ModifiedDate
                    WHERE ProductID = @ProductID";

        // Create a dynamic parameter object with the product properties
        var parameters = new DynamicParameters();
        parameters.Add("@ProductID", product.ProductID);
        parameters.Add("@Name", product.Name);
        parameters.Add("@ProductNumber", product.ProductNumber);
        parameters.Add("@Color", product.Color);
        parameters.Add("@StandardCost", product.StandardCost);
        parameters.Add("@ListPrice", product.ListPrice);
        parameters.Add("@Size", product.Size);
        parameters.Add("@Weight", product.Weight);
        parameters.Add("@ProductCategoryID", product.ProductCategoryID);
        parameters.Add("@ProductModelID", product.ProductModelID);
        parameters.Add("@SellStartDate", product.SellStartDate);
        parameters.Add("@SellEndDate", product.SellEndDate);
        parameters.Add("@DiscontinuedDate", product.DiscontinuedDate);
        parameters.Add("@rowguid", product.rowguid);
        parameters.Add("@ModifiedDate", product.ModifiedDate);

        // Create a connection to the database
        using (var connection = new SqlConnection(_connectionString))
        {
            // Open the connection
            await connection.OpenAsync();

            // Execute the command and return the result as the number of affected rows
            return await connection.ExecuteAsync(sql, parameters);
        }
    }

    // A method that deletes an existing product
    public async Task<int> DeleteProductAsync(int productId)
    {
        // Create a SQL command to delete an existing product
        var sql = @"DELETE FROM Production.Product WHERE ProductID = @ProductID";

        // Create a dynamic parameter object with the product ID
        var parameters = new DynamicParameters();
        parameters.Add("@ProductID", productId);

        // Create a connection to the database
        using (var connection = new SqlConnection(_connectionString))
        {
            // Open the connection
            await connection.OpenAsync();

            // Execute the command and return the result as the number of affected rows
            return await connection.ExecuteAsync(sql, parameters);
        }
    }
}

Notice how we use modular code that is easy to maintain and test, meaningful names for the class, methods, variables, and parameters, concise and clear code that expresses our intent and logic, reusable code that avoids duplication and promotes consistency, and clear flow of execution that follows the single responsibility principle. We also use Dapper’s extension methods, query builders, and custom mappers to simplify our data access code and reduce boilerplate code.

In [6]:
IProductRepository repo = new ProductRepository(connectionString);

Task<IEnumerable<Product>> task =  repo.GetAllProductsAsync();
foreach(var product in task.Result)
{
    Console.WriteLine($"{product.Name}: {product.ListPrice}");
}     

Adjustable Race-6553: 400.0000
Bearing Ball-7693: 0.9900
BB Ball Bearing: 0.0000
Headset Ball Bearings: 0.0000
Blade: 0.0000
LL Crankarm: 0.0000
ML Crankarm: 0.0000
HL Crankarm: 0.0000
Chainring Bolts: 0.0000
Chainring Nut: 0.0000
Chainring: 0.0000
Crown Race: 0.0000
Chain Stays: 0.0000
Decal 1: 0.0000
Decal 2: 0.0000
Down Tube: 0.0000
Mountain End Caps: 0.0000
Road End Caps: 0.0000
Touring End Caps: 0.0000
Fork End: 0.0000
Freewheel: 0.0000
Flat Washer 1: 0.0000
Flat Washer 6: 0.0000
Flat Washer 2: 0.0000
Flat Washer 9: 0.0000
Flat Washer 4: 0.0000
Flat Washer 3: 0.0000
Flat Washer 8: 0.0000
Flat Washer 5: 0.0000
Flat Washer 7: 0.0000
Fork Crown: 0.0000
Front Derailleur Cage: 0.0000
Front Derailleur Linkage: 0.0000
Guide Pulley: 0.0000
LL Grip Tape: 0.0000
ML Grip Tape: 0.0000
HL Grip Tape: 0.0000
Thin-Jam Hex Nut 9: 0.0000
Thin-Jam Hex Nut 10: 0.0000
Thin-Jam Hex Nut 1: 0.0000
Thin-Jam Hex Nut 2: 0.0000
Thin-Jam Hex Nut 15: 0.0000
Thin-Jam Hex Nut 16: 0.0000
Thin-Jam Hex Nut 5: 0.000